In [36]:
import numpy as np
import scipy as sp
from numpy import linalg as LA
import matplotlib.pyplot as plt

# from EMstatPy import *

import time
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

# import ConfigParser
import json
%matplotlib inline
import pandas as pd
import os
import datetime
from copy import deepcopy
import datetime
# ===================================
# Settings
# ===================================
DATA_DIR = './Data/'
PARAMETER_FILE ='Magnet.json'

MAGNET_NAMES = ['CoBarMagnet_z_motion_y']


# determine margin

# determine margin with respect to magnet boundaries for which to calculate fields
# if a margin is none, the margin is set to half the magnet size along that direction
# a1_margin_left, a1_margin_right = 0.25, 0.25
# a2_margin_left, a2_margin_right = None, 1.5
# a3_margin_left, a3_margin_right = 0.25, 0.25

margins = {
    'a1':{'left': None, 'right': 2.0},
    'a2':{'left':0.25, 'right':0.25},
    'a3':{'left':0.25, 'right':0.25}
}

dx = 2e-2 # get a sample every 20nm
# dx = 5e-1 # get a sample every 20nm

# function definitions scalar

In [10]:
# =====================================================================================
def calcSingleMomentBfield(r, ri, m):
    '''
    Calculate the magnetic field of a single magnetic moment

    r:  position in space in nm
    ri: dipole location in space in nm

    m: magnetic moment of a dipole in 1e-18 J / T

    Output in T

    '''
    mu0 = 4 * np.pi *1e-7 # T m /A


    a = r-ri
    rho = np.sqrt(np.vdot(a, a))

    B = mu0 / (4 * np.pi ) * (
         3. * a * np.vdot(m, a) / rho**5
        - m / rho**3
        )


    return B


# =====================================================================================
def calcBfield(r, DipolePositions, m):
    '''
    Calculate the magnetic field for a collection of dipoles
    r:  position in space in nm
    ri: dipole location in space in nm

    m: magnetic moment of a dipole in 1e-18 J/T

    Output in T
    '''

    res = 0.

    for ri in DipolePositions:
        res += calcSingleMomentBfield(r, ri, m)
    return res


def calcSingleMomentGradient(r, ri, m, s, n):
    '''
    Calculate the Gradient of a single magnetic moment

    r:  position in space in nm
    ri: dipole location in space in nm

    m: magnetic moment of a dipole in 1e-18 J / T
    s: spin vector no units
    n: projection vector of the gradient, e.g. motion of resonator

    Output in T/um

    '''
    mu0 = 4 * np.pi *1e-7


    a = r-ri
    rho = np.sqrt(np.vdot(a, a))

    gradB = 3. * mu0 / (4 * np.pi * (rho)**5) * (
         np.vdot(a, m) * np.vdot(s, n)
        +np.vdot(a, s) * np.vdot(m, n)
        - (5 * np.vdot(a, s) * np.vdot(a, m) / rho**2 - np.vdot(m, s) ) * np.vdot(a, n)
        )

    return gradB



# =====================================================================================
def calcGradient(r, DipolePositions, m, s, n):
    '''
    Calculate the gradient for a collection of dipoles
    r:  position in space in nm
    ri: dipole location in space in nm

    m: magnetic moment of a dipole in 1e-18J/T
    s: spin vector no units
    n: projection vector of the gradient, e.g. motion of resonator

    Output in T/um
    '''

    res = 0.

    for ri in DipolePositions:
        res += calcSingleMomentGradient(r, ri, m, s, n)
    return res

# =====================================================================================
def calcDipolePositions(MagnetDimensions, NumberOfDipoles):
    '''
        MagnetDimensions: vector with components height, width, length in um
        NumberOfDipoles: Number of along each dimension, x (height), y (width), z (length)
    '''

    Nx, Ny, Nz = NumberOfDipoles
    H, W, L = MagnetDimensions

    X = np.linspace(-H/2,H/2,Nx, endpoint=True)
    Y = np.linspace(-W/2,W/2,Ny, endpoint=True)
    Z = np.linspace(-L/2,L/2,Nz, endpoint=True)

    DipolePositions = np.array([np.array([x,y,z]) for x in X for y in Y for z in Z])

    return DipolePositions

# =====================================================================================
def calcDipoleMoment(DipolePositions, MagnetDimensions, AtomMass, AtomDensity, gFactor):
    '''
        Calculated the dipole moment of a single dipole in the magnet in units of 1e-18 J / T
        DipolePositions in um
        MagnetDimensions in um
        AtomMass in atomic units (Daltons)
        AtomDensity in 1/m^3
    '''
    BohrMagneton = 9.27e-24 # J/T
    N = AtomDensity/ (AtomMass * 1.66e-27) * np.prod(MagnetDimensions)
    m = gFactor * BohrMagneton * N / len(DipolePositions)

    return m

# function definitions vector

In [14]:
def calcBfield_vec(r, DipolePositions, m):
    '''
    Calculate the magnetic field for a collection of dipoles
    r: (vector of length 3), position in space in nm 
    ri: (matrix with dimension m x 3) m dipole locations in space in nm
    
    m: magnetic moment of a dipole in 1e-18 J/T
    
    Output in T
    '''

    
    
    res = 0.
    
    mu0 = 4 * np.pi *1e-7 # T m /A
    
    a = np.ones((np.shape(DipolePositions)[0],1)) * np.array([r])-DipolePositions
    rho = np.array([np.sqrt(np.sum(a**2,1))]).transpose()*np.ones((1,3))
    # calculate the vector product of m and a: m*(r-ri)
    ma = np.array([np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([m])*a,1)]).transpose()*np.ones((1,3))
    
    B = mu0 / (4 * np.pi ) * ( 3. * a * ma / rho**5
        - m / rho**3
        )
           
    return np.sum(B,0)

In [11]:
def setup_calculation(config, margins):
    """
    config: dictionary with the settings for the magnet
    """
    

    density = float(config['magnet_material_parameters']['density_kg/m^3'])
    atom_mass = float(config['magnet_material_parameters']['atom_mass_dalton'])
    g_factor = float(config['magnet_material_parameters']['g_factor'])

    
    md = config['magnet_dimensions_um']
    magnet_dimensions = np.array([float(md['height']), float(md['width']), float(md['length'])])
    height, width, length = magnet_dimensions
    nd = config['number_of_dipoles']
        
    number_of_dipoles = np.array([float(nd['nx']), float(nd['ny']), float(nd['nz'])])
    dipole_positions = calcDipolePositions(magnet_dimensions, number_of_dipoles)

    s  = np.array(config['unit_vector_spin'])
    m  = np.array(config['unit_vector_magnet'])
    n  = np.array(config['unit_vector_motion'])

    s /= np.linalg.norm(s)
    m /= np.linalg.norm(m)
    n /= np.linalg.norm(n)
    
    m *= calcDipoleMoment(dipole_positions, magnet_dimensions, atom_mass, density, g_factor)
    
    A1, A2, A3 = calc_grid(margins, dx, md)
    
    return A1, A2, A3, s, m, n, dipole_positions
def calc_grid(margins, dx, md):
    """
    margin: margins as a dictionary
    dx: grid spacing betwe
    md: magnet dimensions as dictionary
    """
    magnet_dimensions = np.array([float(md['height']), float(md['width']), float(md['length'])])
    height, width, length = magnet_dimensions
        
    a1min = 0.0 if margins['a1']['left'] is None else -height/2-margins['a1']['left']
    a1max = 0.0 if margins['a1']['right'] is None else height/2+margins['a1']['right']
    a2min = 0.0 if margins['a2']['left'] is None else -width/2-margins['a2']['left']
    a2max = 0.0 if margins['a2']['right'] is None else width/2+margins['a2']['right']
    a3min = 0.0 if margins['a3']['left'] is None else -length/2-margins['a3']['left']
    a3max = 0.0 if margins['a3']['right'] is None else length/2+margins['a3']['right']

    A3 = np.linspace(a3min, a3max,(a3max-a3min)/dx, endpoint=True)
    A2 = np.linspace(a2min, a2max,(a2max-a2min)/dx, endpoint=True)
    A1 = np.linspace(a1min, a1max,(a1max-a1min)/dx, endpoint=True)
    
    return A1, A2, A3

In [12]:
config = json.loads(open('{:s}'.format(PARAMETER_FILE)).read())
A1, A2, A3, s, m, n, dipole_positions = setup_calculation(config[MAGNET_NAMES[0]], margins)

# speed up for B field

In [16]:
a1, a2 = A1[5], A2[4]

rs = np.array([np.array([a1,a2,a3]) for a3 in A3])
rs  = rs[0:4,:]

t1 = datetime.datetime.now()
B = np.array([calcBfield(r, dipole_positions, m) for r in rs])
t2 = datetime.datetime.now()
print('scalar calculation')
print((t2-t1))

t3 = datetime.datetime.now()
B2 = np.array([calcBfield_vec(r, dipole_positions, m) for r in rs])
t4 = datetime.datetime.now()
print('vector calculation')
print((t4-t3))
print('speed up')
print((t2-t1)/(t4-t3))
print('identical result:')
np.sum(B - B2)

scalar calculation
0:00:00.541468
vector calculation
0:00:00.012211
speed up
44.342641880271884
identical result:


0.0

# speed up for Gradient field

In [35]:
a1, a2 = A1[5], A2[4]

rs = np.array([np.array([a1,a2,a3]) for a3 in A3])
# rs  = rs[0:4,:]

t1 = datetime.datetime.now()
G = np.array([calcGradient(r, dipole_positions, m, s, n) for r in rs])
t2 = datetime.datetime.now()
print('scalar calculation')
print((t2-t1))

t3 = datetime.datetime.now()
G2 = np.array([calcGradient_vec(r, dipole_positions, m, s, n) for r in rs])
t4 = datetime.datetime.now()
print('vector calculation')
print((t4-t3))
print('speed up')
print((t2-t1)/(t4-t3))
print('identical result:')
np.sum(G - G2)

scalar calculation
0:00:08.947462
vector calculation
0:00:00.136127
speed up
65.72878268087888
identical result:


6.8828406515897278e-15

In [33]:
# =====================================================================================
def calcGradient_vec(r, DipolePositions, m, s, n):
    '''
    Calculate the gradient for a collection of dipoles
    r: (vector of length 3) position in space in nm
    ri: (matrix with dimension m x 3) m dipole location in space in nm

    m: (vector of length 3) magnetic moment of a dipole in 1e-18J/T
    s: (vector of length 3) spin vector no units
    n: (vector of length 3) projection vector of the gradient, e.g. motion of resonator

    Output in T/um
    '''

    mu0 = 4 * np.pi *1e-7


    a = np.ones((np.shape(DipolePositions)[0],1)) * np.array([r])-DipolePositions
    rho = np.sqrt(np.sum(a**2,1))
    # calculate the vector product of m and a: m*(r-ri)
    ma = np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([m])*a,1)
    # calculate the vector product of s and a: s*(r-ri)
    sa = np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([s])*a,1)
    # calculate the vector product of n and a: n*(r-ri)
    na = np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([n])*a,1)

    
    gradB = 3. * mu0 / (4 * np.pi * (rho)**5) * (
         ma * np.vdot(s, n)
        +sa * np.vdot(m, n)
        - (5 * sa * ma / rho**2 - np.vdot(m, s) ) * na
        )
    return np.sum(gradB,0)

In [31]:
calcGradient_vec(rs[0], dipole_positions, m, s, n)

----
(10240,) (10240,) (10240,)
ccc ()


-0.028323195220016407

In [ ]:
# =====================================================================================
def calcGradient_vec(r, DipolePositions, m, s, n):
    '''
    Calculate the gradient for a collection of dipoles
    r: (vector of length 3) position in space in nm
    ri: (matrix with dimension m x 3) m dipole location in space in nm

    m: (vector of length 3) magnetic moment of a dipole in 1e-18J/T
    s: (vector of length 3) spin vector no units
    n: (vector of length 3) projection vector of the gradient, e.g. motion of resonator

    Output in T/um
    '''

    mu0 = 4 * np.pi *1e-7


    a = np.ones((np.shape(DipolePositions)[0],1)) * np.array([r])-DipolePositions
    rho = np.array([np.sqrt(np.sum(a**2,1))]).transpose()*np.ones((1,3))
    # calculate the vector product of m and a: m*(r-ri)
    ma = np.array([np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([m])*a,1)]).transpose()*np.ones((1,3))
    sa = np.array([np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([s])*a,1)]).transpose()*np.ones((1,3))
    na = np.array([np.sum(np.ones((np.shape(DipolePositions)[0],1)) * np.array([n])*a,1)]).transpose()*np.ones((1,3))
    print('----')
    print(np.shape(ma), np.shape(sa), np.shape(na))
    
    gradB = 3. * mu0 / (4 * np.pi * (rho)**5) * (
         ma * np.vdot(s, n)
        +sa * np.vdot(m, n)
        - (5 * sa * ma / rho**2 - np.vdot(m, s) ) * na
        )
    print('ccc', np.shape(np.sum(gradB,0)))
    return np.sum(gradB,0)